In [3]:
from itertools import chain
latent_lists = [[106, 119], [106, 119], [106, 117, 128, 141], [106], [106, 117], [106, 117, 128, 139], [106], [106, 117], [106, 117, 128], [106, 117], [106, 117], [106, 119, 130, 143, 154], [106, 117, 128, 139], [106], [106, 117], [106, 117, 128], [106, 117], [106], [106], [106, 117], [106, 117, 128, 139], [106, 117], [106], [106, 119, 130, 141, 154, 165], [106, 117], [106, 117, 128], [106, 117, 128], [106, 117, 128], [106, 117], [106, 117, 128], [106, 119, 132, 144], [106, 117, 128]]

indices = sorted(list(set(list(chain.from_iterable(latent_lists)))))
compute_ranges = [(0, indices[0])] + [(indices[i], indices[i+1]) for i in range(len(indices)-1)]
compute_ranges

[(0, 106),
 (106, 117),
 (117, 119),
 (119, 128),
 (128, 130),
 (130, 132),
 (132, 139),
 (139, 141),
 (141, 143),
 (143, 144),
 (144, 154),
 (154, 165)]

In [2]:
compute_ranges

[(0, 106), (106, 117), (117, 128), (128, 139), (139, 154), (154, 165)]

In [4]:
latent_lists.pop(0)

[106, 119]

In [5]:
latent_lists

[[106, 119],
 [106, 117, 128, 141],
 [106],
 [106, 117],
 [106, 117, 128, 139],
 [106],
 [106, 117],
 [106, 117, 128],
 [106, 117],
 [106, 117],
 [106, 119, 130, 143, 154],
 [106, 117, 128, 139],
 [106],
 [106, 117],
 [106, 117, 128],
 [106, 117],
 [106],
 [106],
 [106, 117],
 [106, 117, 128, 139],
 [106, 117],
 [106],
 [106, 119, 130, 141, 154, 165],
 [106, 117],
 [106, 117, 128],
 [106, 117, 128],
 [106, 117, 128],
 [106, 117],
 [106, 117, 128],
 [106, 119, 132, 144],
 [106, 117, 128]]

In [1]:
from utils import BatchComputeRangeIterator, ComputeRange

ins1_ranges = [ComputeRange(0, 78, "lang"), ComputeRange(78, 80, "latent"), ComputeRange(80, 89, "lang"), ComputeRange(89, 91, "latent"), ComputeRange(91, 100, "lang")]
ins2_ranges = [ComputeRange(0, 78, "lang"), ComputeRange(78, 85, "latent"), ComputeRange(85, 90, "lang"), ComputeRange(90, 95, "latent"), ComputeRange(95, 100, "lang")]
ins3_ranges = [ComputeRange(0, 78, "lang"), ComputeRange(78, 90, "latent"), ComputeRange(90, 95, "lang"), ComputeRange(95, 98, "latent"), ComputeRange(98, 100, "lang")]

c_iter = BatchComputeRangeIterator([ins1_ranges, ins2_ranges, ins3_ranges])

In [2]:
for next_ranges in c_iter:
    print(next_ranges)

((ComputeRange(start=0, end=78, mode='lang'), [0, 1, 2]), None)
(None, (ComputeRange(start=78, end=80, mode='latent'), [0, 1, 2]))
((ComputeRange(start=80, end=89, mode='lang'), [0]), (ComputeRange(start=80, end=85, mode='latent'), [1, 2]))
((ComputeRange(start=85, end=90, mode='lang'), [1]), (ComputeRange(start=85, end=90, mode='latent'), [2]))
(None, (ComputeRange(start=89, end=91, mode='latent'), [0]))
((ComputeRange(start=90, end=95, mode='lang'), [2]), (ComputeRange(start=90, end=95, mode='latent'), [1]))
((ComputeRange(start=91, end=100, mode='lang'), [0]), None)
((ComputeRange(start=95, end=100, mode='lang'), [1]), (ComputeRange(start=95, end=98, mode='latent'), [2]))
((ComputeRange(start=98, end=100, mode='lang'), [2]), None)


In [8]:
from datasets import load_dataset, load_from_disk

dataset = load_from_disk("data/competition_math_with_steps_and_flags")

In [9]:
dataset["train"][500]

{'problem': 'A $100$-gon $P_1$ is drawn in the Cartesian plane.  The sum of the $x$-coordinates of the $100$ vertices equals 2009.  The midpoints of the sides of $P_1$ form a second $100$-gon, $P_2$.  Finally, the midpoints of the sides of $P_2$ form a third $100$-gon, $P_3$.  Find the sum of the $x$-coordinates of the vertices of $P_3$.',
 'level': 'Level 5',
 'type': 'Algebra',
 'solution': 'Let the $x$-coordinates of the vertices of $P_1$ be $x_1,x_2,\\ldots,x_{100}$.  Then, by the midpoint formula, the $x$-coordinates of the vertices of $P_2$ are $\\frac{x_1+x_2}2,\\frac{x_2+x_3}2,\\ldots,\\frac{x_{100}+x_1}2 $.  The sum of these equals $\\frac{2x_1+2x_2+\\cdots +2x_{100}}2=x_1+x_2+\\cdots+x_{100}$.  Similarly, the sum of the $x$-coordinates of the vertices of $P_3$ equals the sum of the $x$-coordinates of the vertices of  $P_2$.  Thus the desired answer is $\\boxed{2009}$.',
 'steps': ['Let the $x$-coordinates of the vertices of $P_1$ be $x_1,x_2,\\ldots,x_{100}$.',
  'Then, by th

In [10]:
dataset = dataset["train"]

In [11]:
def extract_answer(solution):
    # extract answer in the form \\boxed{2009}, \\boxed{\\frac{4}{35}}, $x\\equiv -36 \\equiv \\boxed{9} \\pmod {45}$
    import re
    # Find the start of \\boxed{
    match = re.search(r"\\boxed\{", solution)
    if not match:
        return ""
    
    # Count braces to find the matching closing brace (handles nested braces)
    start = match.end()
    depth = 1
    i = start
    while i < len(solution) and depth > 0:
        if solution[i] == '{':
            depth += 1
        elif solution[i] == '}':
            depth -= 1
        i += 1
    
    if depth == 0:
        return solution[start:i-1]
    else:
        return ""

# Add new columns using add_column
dataset = dataset.add_column("question", [item["problem"] for item in dataset])
dataset = dataset.add_column("answer", [extract_answer(item["solution"]) for item in dataset])

# split into train, val and test
dataset = dataset.shuffle(seed=42)
train_size = int(0.8 * len(dataset))
val_size = int(0.1 * len(dataset))
test_size = len(dataset) - train_size - val_size
train_dataset = dataset.select(range(0, train_size))
val_dataset = dataset.select(range(train_size, train_size + val_size))
test_dataset = dataset.select(range(train_size + val_size, len(dataset)))

In [12]:
dataset[1023]

{'problem': 'Let $\\{a_n\\}_{n\\geq 1}$ be an arithmetic sequence and $\\{g_n\\}_{n\\geq 1}$ be a geometric sequence such that the first four terms of $\\{a_n+g_n\\}$ are $0$, $0$, $1$, and $0$, in that order.  What is the next term of $\\{a_n+g_n\\}$?\n\nNote: Duplicate problem',
 'level': 'Level 5',
 'type': 'Intermediate Algebra',
 'solution': 'Since $\\{a_n\\}$ is an arithmetic sequence, we may let $a_n = a + (n-1)d$ for some $a$ and $d.$ Since $\\{g_n\\}$ is a geometric sequence, we may let $g_n = cr^{n-1}$ for some $c$ and $r.$ Then we have \\[\\begin{aligned} a + c &= 0 \\\\ a + d + cr &= 0 \\\\ a + 2d + cr^2 &= 1 \\\\ a + 3d + cr^3 &= 0. \\end{aligned}\\]The first equation gives $c = -a,$ so the remaining equations become \\[\\begin{aligned} a + d - ar &= 0 \\\\ a + 2d - ar^2  &= 1 \\\\ a + 3d - ar^3  &=0. \\end{aligned}\\]From the equation $a+d-ar=0,$ we get $d=ar-a,$ and substituting in the remaining two equations gives \\[\\begin{aligned} -a + 2ar - ar^2 &= 1 \\\\ -2a + 3ar 

In [13]:
train_dataset, val_dataset, test_dataset

(Dataset({
     features: ['problem', 'level', 'type', 'solution', 'steps', 'num_steps', 'split_flags', 'split_warnings', 'split_bad', 'split_needs_resplit', 'question', 'answer'],
     num_rows: 10000
 }),
 Dataset({
     features: ['problem', 'level', 'type', 'solution', 'steps', 'num_steps', 'split_flags', 'split_warnings', 'split_bad', 'split_needs_resplit', 'question', 'answer'],
     num_rows: 1250
 }),
 Dataset({
     features: ['problem', 'level', 'type', 'solution', 'steps', 'num_steps', 'split_flags', 'split_warnings', 'split_bad', 'split_needs_resplit', 'question', 'answer'],
     num_rows: 1250
 }))

In [14]:
import json
import os

# Create directory if it doesn't exist
os.makedirs("data/math", exist_ok=True)

# save datasets as json files (array of objects)
with open("data/math/math_train.json", "w") as f:
    json.dump([item for item in train_dataset], f, indent=2)

with open("data/math/math_val.json", "w") as f:
    json.dump([item for item in val_dataset], f, indent=2)

with open("data/math/math_test.json", "w") as f:
    json.dump([item for item in test_dataset], f, indent=2)